In [1]:
import pandas as pd
import duckdb

In [2]:
df = pd.read_excel(r'1 - Derive Points Table for ICC Tournament.xlsx', usecols = 'A:C', nrows = 5, header = 1 )
df

,Team_1,Team_2,Winner
0,Aus,India,India
1,Eng,NZ,NZ
2,India,SL,India
3,SA,Eng,Eng
4,SL,Aus,Aus


In [3]:
solution1 = duckdb.query(
    """
        ;WITH DistinctTeams AS ( 
        	SELECT Team_1 AS Team FROM df
        	UNION
        	SELECT Team_2 AS Team FROM df
        ),
        Result AS ( 
        	SELECT Team, 
        		(
        			SELECT COUNT(*)
        			FROM (
        				SELECT Team_1 AS Team FROM df
        				UNION ALL
        				SELECT Team_2 AS Team FROM df
        			) AS A2
        			WHERE A1.Team = A2.Team
        		) AS MatchesPlayed,
                
        		(
        			SELECT COUNT(*) 
        			FROM df AS A2
        			WHERE A2.Winner = A1.Team
        		) AS Wins,
                
        		( 
        			SELECT COUNT(*) 
        			FROM df AS A2
        			WHERE (A2.Team_1 = A1.Team OR A2.Team_2 = A1.Team) AND A2.Winner <> A1.Team
        		) AS Losses
        	FROM DistinctTeams AS A1
        )
        
        SELECT * 
        FROM Result
        ORDER BY MatchesPlayed DESC, Wins DESC
    """
).to_df()
solution1

,Team,MatchesPlayed,Wins,Losses
0,India,2,2,0
1,Aus,2,1,1
2,Eng,2,1,1
3,SL,2,0,2
4,NZ,1,1,0
5,SA,1,0,1


In [4]:
solution2 = duckdb.query(
    """
        ;WITH Temp AS (
        	SELECT Team_1 AS Team, 
        		CASE WHEN Team_1 = Winner THEN 1 ELSE 0 END AS Win,
        		CASE WHEN Team_1 <> Winner THEN 1 ELSE 0 END AS Loss
        	FROM df
        	UNION ALL
        	SELECT Team_2 AS Team, 
        		CASE WHEN Team_2 = Winner THEN 1 ELSE 0 END AS Win,
        		CASE WHEN Team_2 <> Winner THEN 1 ELSE 0 END AS Loss
        	FROM df
        )
        
        SELECT  Team, 
        		COUNT(*) AS MatchesPlayed, 
        		SUM(Win) AS Wins, 
        		SUM(Loss) AS Losses
        FROM Temp 
        GROUP BY Team
        ORDER BY MatchesPlayed DESC, Wins DESC
    """
).to_df()

solution2

,Team,MatchesPlayed,Wins,Losses
0,India,2,2.0,0.0
1,Aus,2,1.0,1.0
2,Eng,2,1.0,1.0
3,SL,2,0.0,2.0
4,NZ,1,1.0,0.0
5,SA,1,0.0,1.0
